In [1]:
import os
import json
import pandas as pd
import numpy as np

In [2]:
no_kg_outputs = 'outputs/no_kg_results.jsonl'
text_kg_outputs = 'outputs/text_kg_results.jsonl'
text_kg_1_outputs = 'outputs/text_kg_1_results.jsonl'
text_kg_2_outputs = 'outputs/text_kg_2_results.jsonl'

In [3]:
all_results = []

In [4]:
with open(no_kg_outputs, 'r') as f:
	no_kg_results = [json.loads(line) for line in f.readlines()]
with open(text_kg_outputs, 'r') as f:
	text_kg_results = [json.loads(line) for line in f.readlines()]
with open(text_kg_1_outputs, 'r') as f:
	text_kg_1_results = [json.loads(line) for line in f.readlines()]
with open(text_kg_2_outputs, 'r') as f:
	text_kg_2_results = [json.loads(line) for line in f.readlines()]
with open('/home/jovyan/manish-thesis/thesis-implementations/quest_generation/llama2/data/val.jsonl', 'r') as f:
	val_set = [json.loads(line) for line in f.readlines()]

In [5]:
for no_kg_result, text_kg_result, text_kg_1_result, text_kg_2_result, val in zip(no_kg_results, text_kg_results, text_kg_1_results, text_kg_2_results, val_set):
    result = {
    	'desc' : val['quest']['description'],
		'game' : val['game'],
		'no_kg_input': no_kg_result['input'],
		'text_kg_input': text_kg_result['input'],
		'no_kg': no_kg_result['output_gen'][:-10][11:],
		'text_kg': text_kg_result['output_gen'][:-10][11:],
		'text_1_kg': text_kg_1_result['output_gen'][:-10][11:],
		'text_2_kg': text_kg_2_result['output_gen'][:-10][11:],
  		'actual': no_kg_result['output_actual'][:-10][11:],
	}
    all_results.append(result)

In [6]:
results_df = pd.DataFrame(all_results)

In [7]:
results_df.head()

,desc,game,no_kg_input,text_kg_input,no_kg,text_kg,text_1_kg,text_2_kg,actual
0,Thank you for rescuing me. I had recovered the...,TL2,### Plots:\nthe player freed Elder Josimon fro...,### Background:\nElder Josimon is a character....,Title: The passkey ember\nObjective: Bring eld...,Title: The elder's request\nObjective: Bring e...,Title: The watchweald portal\nObjective: Find ...,Title: The ember keys\nObjective: Find the emb...,Title: Up in smoke\nObjective: Put the slavers...
1,Name's Valdr. I hunt out of Falkreath.\nWe tra...,TESS,"### Plots:\nValdr, Ari and Niels tracked a bea...",### Background:\nValdr is a character. Valdr i...,Title: The spriggans\nObjective: Find the spri...,Title: Moss mother cavern\nObjective: Kill the...,Title: The quest related to the above informat...,Title: The hunt\nObjective: Help valdr track d...,Title: Moss mother cavern\nObjective: Help val...
2,My daughter is gone! I left my home in your ca...,BG2,### Plots:\nGarren Windspear left his home in ...,### Background:\nGarren Windspear is a charact...,Title: The windspear daughter\nObjective: Help...,Title: The windspear family\nObjective: Rescue...,Title: Garren windspear's daughter\nObjective:...,Title: The lost daughter\nObjective: Find garr...,Title: Rescue garren's child from firkraag\nOb...
3,You're really going to the Blue Palace? That p...,TESS,### Plots:\nthe player told Taarie that the pl...,### Background:\nTaarie is a character. Taarie...,Title: A gift for the queen\nObjective: Bring ...,Title: The blue palace\nObjective: Deliver a l...,Title: The blue palace\nObjective: Give jarl e...,Title: The radiant raiment\nObjective: Find th...,Title: Fit for a jarl\nObjective: Promote the ...
4,Why are you people doing this? Is it informati...,TESS,### Plots:\nthe player is torturing their pris...,### Background:\nSolitude is a location. Solit...,Title: The prisoner's inheritance\nObjective: ...,Title: A secret to die for\nObjective: Find th...,Title: The heir to the throne\nObjective: Find...,Title: The hidden inheritance\nObjective: Find...,Title: The inheritance\nObjective: Find the hi...


In [8]:
out_file = os.path.join('outputs', f'all_results.jsonl')
with open(out_file, 'w') as outfile:
	for result in all_results:
		json.dump(result, outfile)
		outfile.write('\n')

In [9]:
import bert_score
from bert_score import BERTScorer
bert_score.__version__

import logging
import transformers
transformers.tokenization_utils.logger.setLevel(logging.ERROR)
transformers.configuration_utils.logger.setLevel(logging.ERROR)
transformers.modeling_utils.logger.setLevel(logging.ERROR)

In [10]:
scorer = BERTScorer(model_type='microsoft/deberta-xlarge-mnli', lang="en")

In [11]:
no_kg_out = [x['no_kg'] for x in all_results]
tree_kg_out = [x['text_kg'] for x in all_results]
tree_1_kg_out = [x['text_1_kg'] for x in all_results]
tree_2_kg_out = [x['text_2_kg'] for x in all_results]
actual_out = [x['actual'] for x in all_results]

In [19]:
_, _, n_t_F1 = scorer.score(no_kg_out, tree_kg_out)
_, _, n_a_F1 = scorer.score(no_kg_out, actual_out)
_, _, t_a_F1 = scorer.score(tree_kg_out, actual_out)

_, _, n_t1_F1 = scorer.score(no_kg_out, tree_1_kg_out)
_, _, n_t2_F1 = scorer.score(no_kg_out, tree_2_kg_out)

_, _, t_t1_F1 = scorer.score(tree_kg_out, tree_1_kg_out)
_, _, t_t2_F1 = scorer.score(tree_kg_out, tree_2_kg_out)
_, _, t1_t2_F1 = scorer.score(tree_1_kg_out, tree_2_kg_out)

_, _, t1_a_F1 = scorer.score(tree_1_kg_out, actual_out)
_, _, t2_a_F1 = scorer.score(tree_2_kg_out, actual_out)

In [20]:
f1_scores_df = pd.DataFrame({'noXtree': n_t_F1,
                             'noXactual': n_a_F1,
                             'noXtree1': n_t1_F1,
                             'noXtree2': n_t2_F1,
                             'treeXactual': t_a_F1,
                             'treeXtree1': t_t1_F1,
                             'treeXtree2': t_t2_F1,
                             'tree1Xactual': t1_a_F1,
                             'tree2Xactual': t2_a_F1,
                            })

In [21]:
f1_scores_df.mean()

noXtree         0.746450
noXactual       0.707393
noXtree1        0.730972
noXtree2        0.694948
treeXactual     0.739215
treeXtree1      0.768611
treeXtree2      0.699570
tree1Xactual    0.727804
tree2Xactual    0.676752
dtype: float32

In [22]:
f1_scores_df.std()

noXtree         0.057196
noXactual       0.050361
noXtree1        0.059993
noXtree2        0.090496
treeXactual     0.067480
treeXtree1      0.071341
treeXtree2      0.085722
tree1Xactual    0.056882
tree2Xactual    0.087490
dtype: float32

In [24]:
f1_scores_df.to_csv('all_bertscores.csv', index=False)